In [1]:
import pickle
from keras.utils import to_categorical
import tensorflow as tf

batch_size = 128
num_classes = 10
epochs = 5

# The data, shuffled and split between train and test sets:
(x_train, y_train) = pickle.load(open('../data/cifar_data.pkl', 'rb'))
print('x_train shape:', x_train.shape)
x_train = x_train.astype('float32')
x_train = x_train / 127.5
x_train = x_train - 1
# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
#y_train = y_train.astype('float32')

test_cnt = int(len(x_train) * 0.2)
x_test = x_train[:test_cnt]
y_test = y_train[:test_cnt]

print('y_train shape:',y_train.shape)

Using TensorFlow backend.


x_train shape: (2000, 32, 32, 3)
y_train shape: (2000, 10)


In [2]:
def model(x, y, is_train=False):
    input_layer = x

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
        inputs=conv1,
        filters=32,
        kernel_size=[3, 3],
        padding="valid",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Convolutional Layer #1
    conv3 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)

    # Convolutional Layer #2
    conv4 = tf.layers.conv2d(
        inputs=conv3,
        filters=64,
        kernel_size=[3, 3],
        padding="valid",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 2304])
    dense1 = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(
        inputs=dense1, rate=0.25, training=is_train)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout1, units=10, activation=tf.nn.relu, name='output')
    loss = tf.losses.softmax_cross_entropy(y, logits)

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return logits, loss, acc


x_ = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name='x_')
y_ = tf.placeholder(tf.int64, shape=[None, 10], name='y_')
is_train_ = tf.placeholder(tf.bool, name='is_train_')

logits, loss, acc = model(x_, y_, is_train_)

train_op = tf.train.RMSPropOptimizer(learning_rate=0.001, epsilon=1e-08, decay=0).minimize(loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

import time

def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
    """Generate a generator that input a group of example in numpy.array and
    their labels, return the examples and labels by the given batchsize.

    Parameters
    ----------
    inputs : numpy.array
        (X) The input features, every row is a example.
    targets : numpy.array
        (y) The labels of inputs, every row is a example.
    batch_size : int
        The batch size.
    shuffle : boolean
        Indicating whether to use a shuffling queue, shuffle the dataset before return.

    Examples
    --------
    >>> X = np.asarray([['a','a'], ['b','b'], ['c','c'], ['d','d'], ['e','e'], ['f','f']])
    >>> y = np.asarray([0,1,2,3,4,5])
    >>> for batch in tl.iterate.minibatches(inputs=X, targets=y, batch_size=2, shuffle=False):
    >>>     print(batch)
    ... (array([['a', 'a'],
    ...        ['b', 'b']],
    ...         dtype='<U1'), array([0, 1]))
    ... (array([['c', 'c'],
    ...        ['d', 'd']],
    ...         dtype='<U1'), array([2, 3]))
    ... (array([['e', 'e'],
    ...        ['f', 'f']],
    ...         dtype='<U1'), array([4, 5]))


    Notes
    -------
    - If you have two inputs, e.g. X1 (1000, 100) and X2 (1000, 80), you can ``np.hstack((X1, X2))
    into (1000, 180) and feed into ``inputs``, then you can split a batch of X1 and X2.
    """
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]

for epoch in range(epochs):
    print("---------------")
    start_time = time.time()
    # train
    train_loss, train_acc, n_batch = 0, 0, 0
    for X_train_a, y_train_a in minibatches(x_train, y_train, batch_size=batch_size):
        sess.run(train_op, feed_dict={x_: X_train_a, y_: y_train_a, is_train_: True})
        err, ac = sess.run([loss, acc], feed_dict={x_: X_train_a, y_: y_train_a, is_train_: False})
        train_loss += err
        train_acc += ac
        n_batch += 1
    print("train loss: %f" % (train_loss / n_batch), "   train acc: %f" % (train_acc / n_batch))
    # test
    test_loss, test_acc, n_batch = 0, 0, 0
    for X_test_a, y_test_a in minibatches(x_train, y_train, batch_size=batch_size):
        err, ac = sess.run([loss, acc], feed_dict={x_: X_test_a, y_: y_test_a,is_train_: False})
        test_loss += err
        test_acc += ac
        n_batch += 1
    print("test loss: %f" % (test_loss / n_batch), "   test acc: %f" % (test_acc / n_batch))

    # print info
    print("Epoch %d of %d took %fs" % (epoch + 1, epochs, time.time() - start_time))

sess.close()

train loss: 2.203480    train acc: 0.226562
test loss: 2.185409    test acc: 0.225000
Epoch 1 of 5 took 11.379812s
train loss: 2.061615    train acc: 0.304167
test loss: 2.124090    test acc: 0.261979
Epoch 2 of 5 took 12.121080s
train loss: 1.969321    train acc: 0.332813
test loss: 1.959129    test acc: 0.326562
Epoch 3 of 5 took 12.013116s
train loss: 1.906683    train acc: 0.363542
test loss: 1.972294    test acc: 0.321875
Epoch 4 of 5 took 13.422395s
train loss: 1.872737    train acc: 0.376563
test loss: 1.961314    test acc: 0.342187
Epoch 5 of 5 took 13.322401s
